In [ ]:
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10


class EDAAnalysis:
    """Класс для проведения EDA анализа датасета книг"""

    def __init__(self, data_path):
        self.data_path = Path(data_path)
        self.df = None
        self.output_dir = self.data_path.parent / "eda_output"
        self.output_dir.mkdir(exist_ok=True)

    def load_data(self):


        self.df = pd.read_csv(self.data_path)
        print(f"Файл: {self.data_path}")
        print(f"Загружено успешно: {len(self.df):,} записей\n")

    def basic_info(self):
        print(f"\nРазмер датасета: {self.df.shape[0]:,} строк x {self.df.shape[1]} столбцов")


        print("\nТипы данных:")
        print(self.df.dtypes)

        print("\nПервые 5 записей:")
        print(self.df.head())

        print("\nПоследние 5 записей:")
        print(self.df.tail())

        print("\nУникальные значения по колонкам:")
        for col in self.df.columns:
            unique_count = self.df[col].nunique()
            unique_pct = (unique_count / len(self.df)) * 100
            print(f"  {col:15} {unique_count:6} уникальных ({unique_pct:5.1f} دست٪)")

        print("\nОписательная статистика (числовые колонки):")
        print(self.df.describe())

    def missing_values_analysis(self):

        print("2. АНАЛИЗ ПРОПУЩЕННЫХ ЗНАЧЕНИЙ")

        missing = self.df.isnull().sum()
        missing_pct = (missing / len(self.df)) * 100
        missing_df = pd.DataFrame({
            'Колонка': missing.index,
            'Пропущено': missing.values,
            'Процент': missing_pct.values
        })
        missing_df = missing_df[missing_df['Пропущено'] > 0].sort_values('Пропущено', ascending=False)

        if len(missing_df) > 0:
            print("\nТаблица пропущенных значений:")
            print(missing_df.to_string(index=False))

            total_missing = missing.sum()
            total_cells = np.prod(self.df.shape)
            print(f"\nВсего пропущенных значений: {total_missing:,} из {total_cells:,} ({(total_missing/total_cells)*100:.2f} دست٪)")
        else:
            print("\nПропущенных значений не обнаружено")

        fig, axes = plt.subplots(1, 2, figsize=(16, 6))
        fig.suptitle('Анализ пропущенных значений', fontsize=16, fontweight='bold')

        if len(missing_df) > 0:
            ax1 = axes[0]
            missing_pct_sorted = (self.df.isnull().sum() / len(self.df) * 100).sort_values(ascending=True)
            missing_pct_sorted = missing_pct_sorted[missing_pct_sorted > 0]
            missing_pct_sorted.plot(kind='barh', ax=ax1, color='coral')
            ax1.set_title('Процент пропусков по колонкам')
            ax1.set_xlabel('Процент пропущенных значений (%)')
            ax1.set_ylabel('Колонка')

            for i, v in enumerate(missing_pct_sorted.values):
                ax1.text(v + 0.1, i, f'{v:.1f} دست٪', va='center')
        else:
            axes[0].text(0.5, 0.5, 'Нет пропущенных значений',
                        ha='center', va='center', fontsize=14)
            axes[0].axis('off')

        ax2 = axes[1]
        sns.heatmap(self.df.isnull(), yticklabels=False, cbar=True,
                   cmap='viridis', ax=ax2, cbar_kws={'label': 'Пропуск (True)'})
        ax2.set_title('Тепловая карта пропущенных значений')
        ax2.set_xlabel('Колонка')

        plt.tight_layout()
        save_path = self.output_dir / "01_missing_values.png"
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()

    def distributions_analysis(self):

        print("\n3. СТАТИСТИКА И РАСПРЕДЕЛЕНИЯ")

        numeric_cols = self.df.select_dtypes(include=[np.number]).columns.tolist()

        print("\nДетальная статистика числовых колонок:")
        for col in numeric_cols:
            data = self.df[col].dropna()
            print(f"\n{col}:")
            print(f"  Среднее: {data.mean():.2f}")
            print(f"  Медиана: {data.median():.2f}")
            print(f"  Ст. отклонение: {data.std():.2f}")
            print(f"  Минимум: {data.min():.2f}")
            print(f"  Максимум: {data.max():.2f}")
            print(f"  Q1 (25 دست٪): {data.quantile(0.25):.2f}")
            print(f"  Q3 (75 دست٪): {data.quantile(0.75):.2f}")
            print(f"  IQR: {data.quantile(0.75) - data.quantile(0.25):.2f}")
            print(f"  Асимметрия (skewness): {data.skew():.2f}")
            print(f"  Эксцесс (kurtosis): {data.kurtosis():.2f}")

        fig, axes = plt.subplots(3, 3, figsize=(18, 14))
        fig.suptitle('Распределения числовых переменных', fontsize=16, fontweight='bold')

        for idx, col in enumerate(numeric_cols):
            row = idx
            data = self.df[col].dropna()

            ax1 = axes[row, 0]
            sns.histplot(data, bins=50, kde=True, ax=ax1, color='steelblue')
            ax1.axvline(data.mean(), color='red', linestyle='--', label=f'Среднее: {data.mean():.2f}')
            ax1.axvline(data.median(), color='green', linestyle='--', label=f'Медиана: {data.median():.2f}')
            ax1.set_title(f'Гистограмма: {col}')
            ax1.legend()

            ax2 = axes[row, 1]
            sns.boxplot(x=data, ax=ax2, color='lightcoral')
            ax2.set_title(f'Box plot: {col}')

            ax3 = axes[row, 2]
            sns.violinplot(x=data, ax=ax3, color='lightgreen')
            ax3.set_title(f'Violin plot: {col}')

        plt.tight_layout()
        save_path = self.output_dir / "02_distributions.png"
        plt.savefig(save_path, dpi=300, bbox_inches='tight')

        plt.close()

        print("\nАнализ категориальных переменных:")
        categorical_cols = self.df.select_dtypes(include=['object']).columns.tolist()

        fig, axes = plt.subplots(1, len(categorical_cols), figsize=(18, 6))
        if len(categorical_cols) == 1:
            axes = [axes]
        fig.suptitle('Топ-10 значений категориальных переменных', fontsize=16, fontweight='bold')

        for idx, col in enumerate(categorical_cols):
            top_values = self.df[col].value_counts().head(10)
            print(f"\nТоп-10 {col}:")
            print(top_values)

            ax = axes[idx]
            top_values.plot(kind='barh', ax=ax, color='teal')
            ax.set_title(f'Топ-10: {col}')
            ax.invert_yaxis()

        plt.tight_layout()
        save_path = self.output_dir / "03_categorical.png"
        plt.savefig(save_path, dpi=300, bbox_inches='tight')

        plt.close()

    def outliers_analysis(self):

        print("\n4. АНАЛИЗ ВЫБРОСОВ И АНОМАЛИЙ")

        numeric_cols = self.df.select_dtypes(include=[np.number]).columns.tolist()

        print("\nОбнаружение выбросов методом IQR:")
        outliers_summary = []

        for col in numeric_cols:
            data = self.df[col].dropna()
            Q1 = data.quantile(0.25)
            Q3 = data.quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            outliers = data[(data < lower_bound) | (data > upper_bound)]
            outliers_count = len(outliers)
            outliers_pct = (outliers_count / len(data)) * 100

            outliers_summary.append({
                'Колонка': col,
                'Нижняя граница': lower_bound,
                'Верхняя граница': upper_bound,
                'Количество выбросов': outliers_count,
                'Процент': outliers_pct
            })

            print(f"\n{col}:")
            print(f"  Диапазон: [{lower_bound:.2f}, {upper_bound:.2f}]")
            print(f"  Выбросов: {outliers_count} ({outliers_pct:.2f} دست٪)")
            if outliers_count > 0:
                print(f"  Min выброс: {outliers.min():.2f}")
                print(f"  Max выброс: {outliers.max():.2f}")

        outliers_df = pd.DataFrame(outliers_summary)
        print("\nСводная таблица выбросов:")
        print(outliers_df.to_string(index=False))

        print("\nПоиск подозрительных паттернов:")

        if 'price' in self.df.columns:
            expensive = self.df[self.df['price'] > 100]
            if len(expensive) > 0:
                print(f"\nКниги с ценой > $100: {len(expensive)}")
                print(expensive[['title', 'author', 'price']].head(10))

        if 'rating' in self.df.columns and 'ratings_count' in self.df.columns:
            suspicious = self.df[(self.df['ratings_count'] == 1) & (self.df['rating'] == 5.0)]
            if len(suspicious) > 0:
                print(f"\nКниги с 1 отзывом и рейтингом 5.0: {len(suspicious)}")
                print("(Потенциально завышенные/ненадежные рейтинги)")

        if 'rating' in self.df.columns:
            perfect_ratings = self.df[self.df['rating'] == 5.0]
            if len(perfect_ratings) > 0:
                perfect_pct = (len(perfect_ratings) / len(self.df)) * 100
                print(f"\nКниги с идеальным рейтингом 5.0: {len(perfect_ratings)} ({perfect_pct:.2f} دست٪)")

        duplicates = self.df.duplicated().sum()
        print(f"\nДубликаты записей: {duplicates}")

        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        fig.suptitle('Визуализация выбросов', fontsize=16, fontweight='bold')

        ax1 = axes[0, 0]
        outlier_counts = [item['Количество выбросов'] for item in outliers_summary]
        outlier_labels = [item['Колонка'] for item in outliers_summary]
        ax1.bar(outlier_labels, outlier_counts, color='crimson')
        ax1.set_title('Количество выбросов по колонкам')
        ax1.set_ylabel('Количество выбросов')
        ax1.tick_params(axis='x', rotation=45)

        if 'price' in self.df.columns:
            ax2 = axes[0, 1]
            ax2.scatter(range(len(self.df)), self.df['price'], alpha=0.4, s=10)
            ax2.set_title('Scatter: Price (выбросы видны сверху)')
            ax2.set_ylabel('Price')
            ax2.set_xlabel('Index')
            ax2.set_yscale('log')

        if 'ratings_count' in self.df.columns and 'rating' in self.df.columns:
            ax3 = axes[1, 0]
            ax3.scatter(self.df['ratings_count'], self.df['rating'], alpha=0.3, s=20)
            ax3.set_title('Ratings Count vs Rating')
            ax3.set_xlabel('Ratings Count')
            ax3.set_ylabel('Rating')
            ax3.set_xscale('log')
            ax3.grid(True, alpha=0.3)

        ax4 = axes[1, 1]
        outlier_pcts = [item['Процент'] for item in outliers_summary]
        ax4.barh(outlier_labels, outlier_pcts, color='orange')
        ax4.set_title('Процент выбросов по колонкам')
        ax4.set_xlabel('Процент (%)')
        ax4.invert_yaxis()

        plt.tight_layout()
        save_path = self.output_dir / "04_outliers.png"
        plt.savefig(save_path, dpi=300, bbox_inches='tight')

        plt.close()

    def correlation_analysis(self):

        print("\n5. КОРРЕЛЯЦИОННЫЙ АНАЛИЗ")

        numeric_cols = self.df.select_dtypes(include=[np.number]).columns.tolist()

        if len(numeric_cols) < 2:
            print("\nНедостаточно числовых колонок для корреляционного анализа")
            return

        corr_matrix = self.df[numeric_cols].corr()

        print("\nМатрица корреляций:")
        print(corr_matrix)

        print("\nСильные корреляции (|r| > 0.5):")
        strong_corr = []
        for i in range(len(corr_matrix.columns)):
            for j in range(i+1, len(corr_matrix.columns)):
                corr_val = corr_matrix.iloc[i, j]
                if abs(corr_val) > 0.5:
                    strong_corr.append({
                        'Переменная 1': corr_matrix.columns[i],
                        'Переменная 2': corr_matrix.columns[j],
                        'Корреляция': corr_val
                    })

        if strong_corr:
            print(pd.DataFrame(strong_corr).to_string(index=False))
        else:
            print("Сильных корреляций не обнаружено (все |r| <= 0.5)")

        fig, axes = plt.subplots(1, 2, figsize=(16, 6))
        fig.suptitle('Корреляционный анализ', fontsize=16, fontweight='bold')

        ax1 = axes[0]
        sns.heatmap(corr_matrix, annot=True, fmt='.3f', cmap='coolwarm',
                   center=0, square=True, ax=ax1, cbar_kws={'label': 'Корреляция'})
        ax1.set_title('Тепловая карта корреляций')

        ax2 = axes[1]
        mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
        sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.3f', cmap='coolwarm',
                   center=0, square=True, ax=ax2, cbar_kws={'label': 'Корреляция'})
        ax2.set_title('Корреляционная матрица (нижний треугольник)')

        plt.tight_layout()
        save_path = self.output_dir / "05_correlations.png"
        plt.savefig(save_path, dpi=300, bbox_inches='tight')

        plt.close()

    def additional_insights(self):


        if 'author' in self.df.columns:
            print("\nТоп-5 авторов по количеству книг:")
            top_authors = self.df['author'].value_counts().head(5)
            for author, count in top_authors.items():
                print(f"  {author}: {count} книг")

        if 'publisher' in self.df.columns:
            print("\nТоп-5 издателей по количеству книг:")
            top_publishers = self.df['publisher'].value_counts().head(5)
            for publisher, count in top_publishers.items():
                print(f"  {publisher}: {count} книг")

        if 'price' in self.df.columns:
            print("\nРаспределение книг по ценовым диапазонам:")
            price_bins = [0, 1, 5, 10, 20, float('inf')]
            price_labels = ['<$1', '$1-5', '$5-10', '$10-20', '>$20']
            self.df['price_range'] = pd.cut(self.df['price'].dropna(), bins=price_bins, labels=price_labels)
            price_dist = self.df['price_range'].value_counts().sort_index()
            for price_range, count in price_dist.items():
                pct = (count / len(self.df)) * 100
                print(f"  {price_range}: {count} книг ({pct:.1f} دست٪)")

        if 'ratings_count' in self.df.columns:
            print("\nРаспределение книг по количеству отзывов:")
            count_bins = [0, 1, 5, 10, 50, float('inf')]
            count_labels = ['1', '2-5', '6-10', '11-50', '>50']
            self.df['count_range'] = pd.cut(self.df['ratings_count'], bins=count_bins, labels=count_labels)
            count_dist = self.df['count_range'].value_counts().sort_index()
            for count_range, count in count_dist.items():
                pct = (count / len(self.df)) * 100
                print(f"  {count_range} отзывов: {count} книг ({pct:.1f} دست٪)")

    def run_full_analysis(self):
        """Запуск полного EDA анализа"""
        self.load_data()
        self.basic_info()
        self.missing_values_analysis()
        self.distributions_analysis()
        self.outliers_analysis()
        self.correlation_analysis()
        self.additional_insights()





def main():
    """Главная функция"""

    data_file = "popular_filled.csv"


    eda = EDAAnalysis(data_file)
    eda.run_full_analysis()


if __name__ == "__main__":
    main()

Файл: popular_filled.csv
Загружено успешно: 12,000 записей


Размер датасета: 12,000 строк x 6 столбцов

Типы данных:
title             object
author            object
publisher         object
price            float64
rating           float64
ratings_count      int64
dtype: object

Первые 5 записей:
                                               title                  author  \
0  Life and Freedom. The autobiography of the for...        Robert Kocharyan   
1  Love yourself tender. A book about self-apprec...        Ольга Примаченко   
2  Crooked House / Скрюченный домишко. Книга для ...         Agatha Christie   
3  The Richest Man in Babylon / Самый богатый чел...  Джордж Сэмюэль Клейсон   
4  Алиса в Стране чудес / Alice’s Adventures in W...           Lewis Carroll   

     publisher  price  rating  ratings_count  
0  Альпина ПРО   5.54    5.00              2  
1      БОМБОРА   4.93    4.71             14  
2          NaN   2.89    4.48             23  
3          NaN   3.44    5.00 